**Challenge: Implement a Multiclass Classification Neural Network using PyTorch**

Objective:
Build a feedforward neural network using PyTorch to predict the species of iris flowers in a multiclass classification problem. The dataset used for this challenge is the Iris dataset, which consists of features like sepal length, sepal width, petal length, and petal width.

Steps:

1. **Data Preparation**: Load the MNIST dataset using ```torchvision.datasets.MNIST```. Standardize/normalize the features. Split the dataset into training and testing sets using, for example, ```sklearn.model_selection.train_test_split()```. **Bonus scores**: *use PyTorch's built-* ```DataLoader``` *to split the dataset*.

2. **Neural Network Architecture**: Define a simple feedforward neural network using PyTorch's ```nn.Module```. Design the input layer to match the number of features in the MNIST dataset and the output layer to have as many neurons as there are classes (10). You can experiment with the number of hidden layers and neurons to optimize the performance. **Bonus scores**: *Make your architecture flexibile to have as many hidden layers as the user wants, and use hyperparameter optimization to select the best number of hidden layeres.*

3. **Loss Function and Optimizer**: Choose an appropriate loss function for multiclass classification. Select an optimizer, like SGD (Stochastic Gradient Descent) or Adam.

4. **Training**: Write a training loop to iterate over the dataset.
Forward pass the input through the network, calculate the loss, and perform backpropagation. Update the weights of the network using the chosen optimizer.

5. **Testing**: Evaluate the trained model on the test set. Calculate the accuracy of the model.

6. **Optimization**: Experiment with hyperparameters (learning rate, number of epochs, etc.) to optimize the model's performance. Consider adjusting the neural network architecture for better results. **Notice that you can't use the optimization algorithms from scikit-learn that we saw in lab1: e.g.,** ```GridSearchCV```.


Solaire of Astora here, \\[T]/

This is **not** my first rodeo, not to brag but I myself have trained a lot of models soo... let's see if I can make a massacre out of MNIST for the hundredth time.

Let's first import our tools.

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T # We'll need this one for a lil party trick.
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset, Dataset

import random
import numpy as np

First let's prepare our data, we split into train, valid, and test of course (using seed for reproducibility)

In [9]:
underlying_dataset = MNIST("downloads", train=True, download=True, transform=T.ToTensor())

indexes = list(range(len(underlying_dataset)))
random.seed(42)
random.shuffle(indexes)

n_train = int(len(indexes) * .9)
underlying_train_dset = Subset(underlying_dataset, indexes[:n_train])
underlying_valid_dset = Subset(underlying_dataset, indexes[n_train:])

underlying_test_dset = MNIST("downloads", train=False, download=True, transform=T.ToTensor())

print(n_train, len(indexes) - n_train, len(underlying_test_dset))

54000 6000 10000


As I want to normalize the inputs, let's first calculate mean and standard deviation from the training set.

In [10]:
# images = []
# for img, _ in underlying_train_dset:
#     images.append(img)
#
# images = torch.stack(images)
#
# print(images.mean(), images.std())
## tensor(0.1306) tensor(0.3081)

tensor(0.1306) tensor(0.3081)


In [ ]:
class MyDataset(Dataset):
    def __init__(self, underlying_data, transform):
        self.data = underlying_data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image, label = self.data[idx]
        image = self.transform(image)
        return (image, label)